In [ ]:
import os
import json
import time

from utils import init_spark, remove_accents, to_snake_case, value_splitter
from airport_api import AirportApi

import pandas as pd

from pyspark.sql.functions import col, to_timestamp
from pyspark.sql.types import *

### Diretórios dos dados
extrair conteúdos de `AIR_CIA.zip` e `VRA.zip` (ambos com subpastas) na pasta `./raw_data` ou no diretório relativo em `relative_raw_data_dir`

In [ ]:
relative_raw_data_dir = '../raw_data'
relative_air_cia_dir = f'{relative_raw_data_dir}/AIR_CIA'
relative_vra_dir = f'{relative_raw_data_dir}/VRA'

# Local para salvar tabelas processadas
target_processed_data_dir = '../processed_data'
processed_air_cia_dir = f'{target_processed_data_dir}/air_cia/air_cia.parquet'
processed_vra_dir = f'{target_processed_data_dir}/vra/vra.parquet'
processed_aerodromo_dir = f'{target_processed_data_dir}/aerodromo/aerodromo.parquet'

view_airline_origin_destiny_dir = f'{target_processed_data_dir}/airline_origin_destiny_view/airline_origin_destiny_view.parquet'
view_airport_airline_dir = f'{target_processed_data_dir}/airport_airline_view/airline_origin_destiny_view.parquet'

# Local para queries de views
views_dir = '../views'
query_airline_origin_destiny_dir = f'{views_dir}/airline_origin_destiny.sql'
query_airport_airline_dir = f'{views_dir}/airport_airline.sql'

### Inicialização do PySpark

In [ ]:
spark = init_spark()

In [ ]:
processed_air_cia = raw_air_cia = spark.read.options(header = True, delimiter=';').csv(relative_air_cia_dir)
processed_vra = raw_vra = spark.read.json(relative_vra_dir)

### Mudar nomes de coluna para snake case

In [ ]:
for column_name in raw_air_cia.columns:
    processed_air_cia = processed_air_cia.withColumnRenamed(column_name, to_snake_case(remove_accents(column_name)))

for column_name in raw_vra.columns:
    processed_vra = processed_vra.withColumnRenamed(column_name, to_snake_case(remove_accents(column_name)))

## Procedimentos VRA

### `icao_empresa_aerea` para `iata_empresa_aerea`
Os valores desta coluna, na verdade estão listando os códigos IATA, é possível verificar isso através da chamada de API, quando o mesmo código não existe no índice de ICAO.

In [ ]:
processed_vra = processed_vra.withColumnRenamed('icao_empresa_aerea', 'iata_empresa_aerea')

### Colunas de timestamp para `DateType`

In [ ]:
processed_vra = processed_vra.withColumn('partida_prevista', to_timestamp(col('partida_prevista')))\
    .withColumn('chegada_prevista', to_timestamp(col('chegada_prevista')))\
    .withColumn('partida_real', to_timestamp(col('partida_real')))\
    .withColumn('chegada_real', to_timestamp(col('chegada_real')))

In [ ]:
processed_vra.show(10)

In [ ]:
processed_vra.write.format('parquet').mode('overwrite').save(processed_vra_dir)

## Procedimentos AIR_CIA

### Separação de colunas `icao_iata`

In [ ]:
processed_air_cia = processed_air_cia.withColumn('iata', value_splitter(col('icao_iata'))[0])\
    .withColumn('icao', value_splitter(col('icao_iata'))[1])\
        .drop(col('icao_iata'))

In [ ]:
processed_air_cia.show(10)

In [ ]:
processed_air_cia.write.format('parquet').mode('overwrite').save(processed_air_cia_dir)

## Procedimentos aerodromos

In [ ]:
processed_vra = spark.read.parquet(processed_vra_dir)

### Coletar IATA e ICAO

In [ ]:
key = "my-rapidapi-key" # Adiquirir chave para API em https://rapidapi.com/Active-api/api/airport-info
host = "airport-info.p.rapidapi.com"

api = AirportApi(key, host)

In [ ]:
collected_icao = []

for row in processed_vra.select('icao_aerodromo_origem', 'icao_aerodromo_destino', 'iata_empresa_aerea').collect():
    if row['icao_aerodromo_origem'] not in collected_icao:
        collected_icao.append(row['icao_aerodromo_origem'])
    if row['icao_aerodromo_destino'] not in collected_icao:
        collected_icao.append(row['icao_aerodromo_destino'])

In [ ]:
icao_data = []

for icao in collected_icao:
    this_icao = api.get_airport_info_by_icao(icao)
    this_icao = json.loads(this_icao)

    if this_icao.get('error', None):
        this_icao['icao'] = icao
        
    icao_data.append(this_icao)

In [ ]:
icao_dataframe = pd.DataFrame(data = icao_data)

In [ ]:
filtered_airport = icao_dataframe[icao_dataframe.error != {'text': 'No airport found'}]
not_found_airport = icao_dataframe[icao_dataframe.error == {'text': 'No airport found'}]

filtered_airport['id'] = filtered_airport['id'].astype(int)
filtered_airport['longitude'] = filtered_airport['longitude'].astype(float)
filtered_airport['latitude'] = filtered_airport['latitude'].astype(float)

In [ ]:
aerodromo_schema = StructType([
	StructField('id', IntegerType(), True),
	StructField('iata', StringType(), True),
	StructField('icao', StringType(), True),
	StructField('name', StringType(), True),
	StructField('location', StringType(), True),
	StructField('street_number', StringType(), True),
	StructField('street', StringType(), True),
	StructField('city', StringType(), True),
	StructField('county', StringType(), True),
	StructField('state', StringType(), True),
	StructField('country_iso', StringType(), True),
	StructField('country', StringType(), True),
	StructField('postal_code', StringType(), True),
	StructField('phone', StringType(), True),
	StructField('latitude', DoubleType(), True),
	StructField('longitude', DoubleType(), True),
	StructField('utc', IntegerType(), True),
	StructField('websiite', StringType(), True)
])

aerodromo = spark.createDataFrame(filtered_airport)
aerodromo = aerodromo.drop(col('error'))

In [ ]:
aerodromo.show(10)

In [ ]:
aerodromo.write.format('parquet').mode('overwrite').save(processed_aerodromo_dir)

## Views

In [ ]:
aerodromo = spark.read.parquet(processed_aerodromo_dir)
vra = spark.read.parquet(processed_vra_dir)
air_cia = spark.read.parquet(processed_air_cia_dir)

aerodromo.createOrReplaceTempView('aerodromo')
vra.createOrReplaceTempView('vra')
air_cia.createOrReplaceTempView('air_cia')

### Origem-destino, mais frequente por empresa aérea

In [ ]:
origin_destiny_most_frequence = open(query_airline_origin_destiny_dir).read()

try:
    airline_origin_destiny_view = spark.sql(origin_destiny_most_frequence)
    airline_origin_destiny_view.show(10, truncate = False)
except Exception as e:
    print(e)

### Contagem total de decolagens e pouso no Aeroporto com a empresa mais frequênte
Totaliza o número de decolagens e pousos realizados no aeroporto pela empresa aérea que mais atuou no mesmo.

Contabiliza o total de vezes que o aeroporto foi usado como origem ou destino pela empresa; é possível filtrar por voos realizados para totalizar o número de decolagens e pousos reais.

In [ ]:
airport_to_airline_relation = open(query_airport_airline_dir).read()

try:
    airport_airline_view = spark.sql(airport_to_airline_relation)
    airport_airline_view.show(10, truncate = False)
except Exception as e:
    print(e)

In [ ]:
airline_origin_destiny_view.write.format('parquet').mode('overwrite').save(view_airline_origin_destiny_dir)
airport_airline_view.write.format('parquet').mode('overwrite').save(view_airport_airline_dir)